In [158]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle


In [159]:
train_raw_data =np.loadtxt('/Users/Nagarwa/AI/Kaggle/digit-recognizer/train.csv',delimiter=',')

In [160]:
unscaled_inp = train_raw_data[:,1:]
target_all =train_raw_data[:,0]

In [161]:
shuffle_indices = np.arange(unscaled_inp.shape[0])
np.random.shuffle(shuffle_indices)

shuffle_inputs = unscaled_inp[shuffle_indices]
shuffle_targets = target_all[shuffle_indices]

In [162]:
sample_count = shuffle_inputs.shape[0]

In [163]:
train_sample_count = int(0.9*sample_count)
validation_sample_count = int(0.1*sample_count)

In [164]:
train_inputs = shuffle_inputs[:train_sample_count]
train_target = shuffle_targets[:train_sample_count]

In [165]:
validation_inputs = shuffle_inputs[train_sample_count:]
validation_target = shuffle_targets[train_sample_count:]

In [166]:
test_raw_data =np.loadtxt('/Users/Nagarwa/AI/Kaggle/digit-recognizer/test.csv',delimiter=',')



In [167]:
np.savez('digit_recognizer_train',inputs = train_inputs,targets = train_target)
np.savez('digit_recognizer_valid',inputs = validation_inputs,targets = validation_target)
np.savez('digit_recognizer_test',inputs = test_raw_data)

In [168]:

npz = np.load('digit_recognizer_train.npz')

train_input_ = npz['inputs'].astype(np.int)
train_target_ = npz['targets'].astype(np.int)


npz = np.load('digit_recognizer_valid.npz')

validation_input_ = npz['inputs'].astype(np.int)
validation_target_ = npz['targets'].astype(np.int)

In [169]:
validation_target_

array([7, 5, 7, ..., 9, 3, 1])

In [170]:
inputsize = 783
osize = 10
hidden_layer_size = 50

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
     tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(osize,activation='softmax')    
])

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [171]:
NUM_EPOCHS = 100
batch_size = 100

early_stopping = tf.keras.callbacks.EarlyStopping()

model.fit(train_input_,
         train_target_,
         batch_size=batch_size,
         epochs=NUM_EPOCHS,
          callbacks=[early_stopping],
            validation_data=(validation_input_,validation_target_),
         verbose=2)

Train on 37800 samples, validate on 4200 samples
Epoch 1/100
37800/37800 - 2s - loss: 2.3423 - accuracy: 0.5957 - val_loss: 0.7327 - val_accuracy: 0.7760
Epoch 2/100
37800/37800 - 2s - loss: 0.5820 - accuracy: 0.8218 - val_loss: 0.5178 - val_accuracy: 0.8495
Epoch 3/100
37800/37800 - 2s - loss: 0.4083 - accuracy: 0.8772 - val_loss: 0.3893 - val_accuracy: 0.8876
Epoch 4/100
37800/37800 - 2s - loss: 0.3208 - accuracy: 0.9050 - val_loss: 0.3421 - val_accuracy: 0.8957
Epoch 5/100
37800/37800 - 2s - loss: 0.2688 - accuracy: 0.9196 - val_loss: 0.2763 - val_accuracy: 0.9198
Epoch 6/100
37800/37800 - 2s - loss: 0.2316 - accuracy: 0.9312 - val_loss: 0.2556 - val_accuracy: 0.9248
Epoch 7/100
37800/37800 - 2s - loss: 0.2111 - accuracy: 0.9379 - val_loss: 0.2396 - val_accuracy: 0.9302
Epoch 8/100
37800/37800 - 2s - loss: 0.1835 - accuracy: 0.9449 - val_loss: 0.2331 - val_accuracy: 0.9310
Epoch 9/100
37800/37800 - 2s - loss: 0.1778 - accuracy: 0.9457 - val_loss: 0.2223 - val_accuracy: 0.9357
Epoch 

In [172]:
np_test = np.load('digit_recognizer_test.npz')
np_test_input =  np_test['inputs'].astype(np.int)


cnn_predictions = model.predict(np_test_input)


In [173]:

cnn_predictions


array([[1.0793182e-18, 3.4142806e-12, 9.9991119e-01, ..., 9.0307076e-06,
        2.7616496e-11, 1.5397640e-13],
       [9.9932277e-01, 1.8046094e-13, 3.5581416e-09, ..., 1.5339956e-06,
        3.4921186e-08, 6.5904623e-04],
       [6.1069436e-08, 1.0343804e-09, 1.1913972e-07, ..., 9.5134339e-04,
        1.9819900e-06, 9.9888617e-01],
       ...,
       [5.4035816e-14, 1.3755123e-09, 1.7151159e-07, ..., 5.2393770e-11,
        7.9779362e-05, 1.2483244e-03],
       [3.2995757e-07, 1.9370095e-08, 3.1338437e-05, ..., 1.0326782e-05,
        5.5978580e-07, 9.6325409e-01],
       [4.0083968e-16, 8.7636778e-09, 9.9931586e-01, ..., 2.9359953e-04,
        2.6578313e-08, 3.1114885e-12]], dtype=float32)

In [189]:
prediction=tf.argmax(cnn_predictions,1)

In [197]:
type(prediction)

tensorflow.python.framework.ops.EagerTensor

In [205]:
predicted_output = prediction.numpy()

In [207]:
np.savetxt("Digit_Recognizer_prediction.csv", np.dstack((np.arange(1, predicted_output.size+1),predicted_output))[0],"%d,%d",header="ImageId,Label")